<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Newton_Method_for_Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn,optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [260]:
# Creating our own ConvNet
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 1000)
        self.fc2 = nn.Linear(1000, 10)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)

        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = ConvNet().to(device)
torch.nn.init.xavier_uniform_(model.conv1.weight)
torch.nn.init.xavier_uniform_(model.conv2.weight)


Parameter containing:
tensor([[[[-7.1508e-03,  2.0774e-02, -9.8402e-02,  3.4959e-02,  5.1320e-02],
          [-9.0630e-02,  3.0374e-02,  5.6665e-02,  9.9510e-02, -9.4949e-02],
          [ 2.7301e-02,  5.5158e-02, -1.7174e-03,  6.9166e-02,  4.2873e-02],
          [-7.9089e-03,  2.2498e-02, -9.8922e-02,  4.0853e-02,  3.8763e-02],
          [ 2.5419e-02, -2.8185e-02, -4.5330e-02,  5.6819e-02, -4.2291e-02]],

         [[ 2.3483e-02, -7.7437e-02, -5.4352e-02, -2.2201e-02, -3.2798e-02],
          [-1.0356e-01, -1.8685e-02,  1.3590e-02,  4.8577e-02, -1.2109e-02],
          [-6.2203e-02, -8.3368e-02,  7.4869e-03, -9.3975e-02,  8.1583e-03],
          [ 2.3188e-02,  3.1507e-02, -8.4653e-02,  9.6460e-02, -1.1978e-02],
          [-9.0852e-02, -7.8338e-02, -3.1433e-02,  4.3965e-02, -1.0215e-01]],

         [[ 5.1529e-02,  2.9017e-02,  7.5265e-02, -3.3126e-03,  1.7804e-02],
          [-3.4060e-02,  8.5635e-02,  1.8158e-03,  8.7742e-02, -9.8412e-02],
          [-9.5797e-02,  2.4514e-02, -5.0812e-02, 

In [255]:
from torchvision import transforms
from torchvision.datasets import CIFAR10
train_transforms = transforms.Compose([ transforms.ToTensor(), transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])
train_data = CIFAR10(root="./train/", train=True, download=True, transform=train_transforms)
trainloader = torch.utils.data.DataLoader( train_data, batch_size=512,shuffle=True)

test_data = CIFAR10(root="./test/", train=False, download=True, transform=train_transforms)
testloader = torch.utils.data.DataLoader( test_data,batch_size=512, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [261]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.LBFGS(model.parameters(),lr=0.39,max_iter=4,history_size=10,line_search_fn='strong_wolfe')



In [262]:
N_EPOCHS = 5
for epoch in range(N_EPOCHS):
  epoch_loss = 0.0
  i = 0
  model.train()
  for inputs, labels in trainloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    def closure():
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward(retain_graph=True)
      global epoch_loss
      epoch_loss += loss.item()
  #    print(loss.item())
      return loss

    optimizer.step(closure)



  val_loss = 0.0
  model.eval()
  for inputs, labels in testloader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    loss = criterion(outputs, labels)
    val_loss += loss.item()
  print("Epoch: {} Train Loss: {} Val Loss: {} ".format(epoch, epoch_loss/len(trainloader), val_loss/len(testloader)))



Epoch: 0 Train Loss: 4629.119049381236 Val Loss: 1.6341470539569856 
Epoch: 1 Train Loss: 10.338928778560794 Val Loss: 1.6306005775928498 
Epoch: 2 Train Loss: 7.531226352769501 Val Loss: 1.6276377320289612 
Epoch: 3 Train Loss: 8.307211396645526 Val Loss: 1.6254245519638062 
Epoch: 4 Train Loss: 9.13040437625379 Val Loss: 1.6163111627101898 


In [263]:
y_pred = model(test_data[9][0].unsqueeze(0).to(device))
print(f" Model predicted label : {np.argmax(y_pred.detach().cpu().numpy())}")
print(f" Actual label : {test_data[9][1]}")

 Model predicted label : 1
 Actual label : 1
